In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 载入数据集
mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data", one_hot=True)

# 运行次数
max_steps = 550 * 21

# 图片数量
image_num = 3000

# 定义会话
sess = tf.Session()

# 文件路径
DIR = "C:/Tensorflow/"

# 载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), 
                        trainable=False, name="embedding")

# 定义一个参数概要
def varible_summaries(var):
    
    with tf.name_scope("summary"):
        
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean) # 平均值
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev) # 标准差
        tf.summary.scalar("max", tf.reduce_max(var)) #最大值
        tf.summary.scalar("min", tf.reduce_min(var)) # 最小值
        tf.summary.histogram("histogram", var) # 直方图

# 命名空间
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")        

# 显示图片
with tf.name_scope("input_reshape"):
    
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image("input", image_shaped_input, 10)
    

with tf.name_scope("layer"):
    #创建一个简单的神经网络
    with tf.name_scope("wights1"):
        W1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1), name="W1")
        varible_summaries(W1)
        
    with tf.name_scope("biases1"):
        b1 = tf.Variable(tf.zeros([500]) + 0.1, name="b1")
        varible_summaries(b1)
        
#     with tf.name_scope("wx_plus_b1"):
#         wx_plus_b1 = tf.matmul(x, W1) + b1
    
    with tf.name_scope("L1"):
        L1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
    
    with tf.name_scope("wights2"):
        W2 = tf.Variable(tf.truncated_normal([500, 10], stddev=0.1), name="W2")
        varible_summaries(W2)
        
    with tf.name_scope("biases2"):
        b2 = tf.Variable(tf.zeros([10]) + 0.1, name="b2")
        varible_summaries(b2)
        
    with tf.name_scope("wx_plus_b2"):
        wx_plus_b2 = tf.matmul(L1, W2) + b2
        
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b2) # 预测值

# 二次代价函数

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
with tf.name_scope("loss"):
    loss = loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar("loss", loss)
    
# 梯度下降法
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()
sess.run(init)

with tf.name_scope("accuracy"):
    # 结果存放在一个布尔型列表中
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax返回1维张量中最大的值所在的位置
    # 求准确率
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#cast转换类型，True->1.0, False->0.0
        tf.summary.scalar("accuracy", accuracy)

# 产生 metadata文件        
if tf.gfile.Exists(DIR + "projector/projector/metadata.tsv"):
    tf.gfile.DeleteRecursively(DIR + "projector/projector/metadata.tsv")

with open(DIR + "projector/projector/metadata.tsv", "w") as f:
    lables = sess.run(tf.argmax(mnist.test.labels[:], 1))
    for i in range(image_num):
        f.write(str(lables[i]) + "\n")

# 合并所有的summary
merged = tf.summary.merge_all()


projector_writer = tf.summary.FileWriter(DIR + "projector/projector", sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + "projector/projector/metadata.tsv"
embed.sprite.image_path = DIR + "projector/data/mnist_10k_sprite.png"
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer, config)
    

for i in range(max_steps):

    batch_xs, batch_ys = mnist.train.next_batch(100) #类似于read，一次读取100张图片
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary = sess.run([train_step, merged], feed_dict={x : batch_xs, y : batch_ys})[1]
    projector_writer.add_run_metadata(run_metadata, "step%03d" % i)
    projector_writer.add_summary(summary, i)
    
    if i % 550 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("第", i, "个周期", "准确率是", acc)

saver.save(sess, DIR + "projector/projector/a_model.ckpt")
projector_writer.close()
sess.close()

Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

第 0 个周期 准确率是 0.0913
第 550 个周期 准确率是 0.8161
第 1100 个周期 准确率是 0.8317
第 1650 个周期 准确率是 0.8383
第 2200 个周期 准确率是 0.9278
第 2750 个周期 准确率是 0.9331
第 3300 个周期 准确率是 0.9383
第 3850 个周期 准确率是 0.941
第 4400 个周期 准确率是 0.9433
第 4950 个周期 准确率是 0.9455
第 5500 个周期 准确率是 0.9466
第 6050 个周期 准确率是 0.9485
第 6600 个周期 准确率是 0.9499
第 7150 个周期 准确率是 0.9506
第 7700 个周期 准确率是 0.953
第 8250 个周期 准确率是 0.9539
第 8800 个周期 准确率是 0.9557
第 9350 个周期 准确率是 0.9562
第 9900 个周期 准确率是 0.9569
第 10450 个周期 准确率是 0.9575
第 11000 个周期 准确率是 0